# HER2 One Scanner - Aperio FDA

- 3-Fold (50/50) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [98]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.metrics import confusion_matrix
from collections import Counter
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [99]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [100]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [101]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,z

In [102]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
#print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241


### Old Code

In [103]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [104]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [105]:
#Need to remove random_state before running multiple thousands of times and averaging
kf = KFold(n_splits = 2, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


for train_index_full, test_index in kf.split(y):
    val_index = np.random.choice(train_index_full, 21, replace=False)
    train_index = [x for x in train_index_full if x not in val_index]
    X_train, X_val, X_test = X[train_index], X[val_index], X[test_index]
    y_train, y_val, y_test = y[train_index], y[val_index], y[test_index]
    z_train, z_val, z_test = z[train_index], z[val_index], z[test_index]

2
KFold(n_splits=2, random_state=5, shuffle=True)


In [106]:
len(train_index)
#len(test_index)

100

In [161]:

oos_y = []
oos_pred = []
fold = 0


for train_index_full, test_index in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
    
    random.shuffle(train_index_full)
    random.shuffle(test_index)
    
    val_number = np.rint(len(train_index_full)*0.1)
    print(val_number, type(val_number))
    val_index = np.random.choice(train_index_full, int(val_number), replace=False)
    train_index = [x for x in train_index_full if x not in val_index]
    X_train, X_val, X_test = X[train_index], X[val_index], X[test_index]
    y_train, y_val, y_test = y_cat[train_index], y_cat[val_index], y_cat[test_index]
    ylab_train, ylab_val, ylab_test = y[train_index], y[val_index], y[test_index]
    z_train, z_val, z_test = z[train_index], z[val_index], z[test_index]
    
    
    #Desired Augmentations
    datagen = ImageDataGenerator(
        fill_mode='constant',
        horizontal_flip=True,
        vertical_flip=True,
        )
    
    #Dataset to receive augmentations (augX_train & augX_val)
    by_train = y_train[:,0] == 1
    by_val = y_val[:,0] == 1
    by_test = y_test[:,0] == 1
    augX_train, augX_val, augX_test = X_train[by_train], X_val[by_val], X_test[by_test]
    augy_train, augy_val, augy_test = y_train[by_train], y_val[by_val], y_test[by_test]
    
    #Apply augmentations to augX_train & augX_val:
    augtrainbatch_size = int((len(augX_train))*4)
    print(augtrainbatch_size)
    
    
    #for x in [0,1,2,3,4,5,6,7,8,9]:
        #datagen.flow(x, batch_size=1)
        #datagen.flow(augX_train[:x], batch_size=10)
        #break
        
        #augimX_train = datagen.flow(x, batch_size=1)
    
    augimX_train = datagen.flow(augX_train, augy_train, batch_size=augtrainbatch_size)
    #augimX_train = [for _ in datagen.flow(augX_train, augy_train, batch_size=augtrainbatch_size)]
    print("augX_train:", len(augX_train))
    print(len(augX_train))
    print("auginX_train:", len(augimX_train))
    print(len(augimX_train))
    print(augimX_train)
    #plt.imshow(augimX_train[:,0])
    
    #for _ in datagen.flow(augX_val):
   
    #Concatonate (augfullX_train -> X_train) & (augfullX_val -> X_val):
    
    
    #Model:
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
   
    
    model.fit(
            X_train,
            y_train,
            validation_data=(X_val,y_val),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    #pred = np.argmax(pred)
    predx = np.argmax(pred,axis=1)
    oos_pred.append(predx)
    
    #measure the fold's accuracy:
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, predx)
    print("Fold Score (accuracy): {}".format(score))
    
    #Create confusion matrix:
    print(confusion_matrix(ylab_test, predx))
    
    #Check my work:
    print(y_test)
    #print(test)
    print(pred)
    print(predx)
    #print(z[test])
    #print(oos_y)
    
    
    #outlist = []
    
    #outline = index[0] + '\t' + str(z[test]) + '\t' + str(int(pred[0])) + '\t' + str(int(pred[1])) + '\t' + str(int(pred[2])) + '\t' + str(int(y[test])) + '\t' + str(predx) + \n'
    #outlist.append(outline)
    #with open("results.txt", "w") as f:
        #f.writelines(outlist)
        #f.writelines(z[test])

fold #1
(12.0, <type 'numpy.float64'>)
36
('augX_train:', 9)
9
('auginX_train:', 1)
1
Train on 108 samples, validate on 12 samples
Epoch 1/1000
108/108 [==============================] - 2s 20ms/step - loss: 1.1538 - acc: 0.3796 - val_loss: 0.9855 - val_acc: 0.6667
Epoch 2/1000
108/108 [==============================] - 0s 2ms/step - loss: 1.0152 - acc: 0.4630 - val_loss: 0.8559 - val_acc: 0.6667
Epoch 3/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.9770 - acc: 0.5370 - val_loss: 0.9621 - val_acc: 0.6667
Epoch 4/1000
108/108 [==============================] - 0s 2ms/step - loss: 1.0507 - acc: 0.5741 - val_loss: 1.4296 - val_acc: 0.2500
Epoch 5/1000
108/108 [==============================] - 0s 2ms/step - loss: 1.2101 - acc: 0.4352 - val_loss: 0.7701 - val_acc: 0.6667
Epoch 6/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.8940 - acc: 0.6204 - val_loss: 0.9034 - val_acc: 0.7500
Epoch 7/1000
108/108 [==============================] - 0s 2ms/s

108/108 [==============================] - 0s 2ms/step - loss: 0.3319 - acc: 0.8981 - val_loss: 0.9952 - val_acc: 0.6667
Epoch 00060: early stopping
Fold Score (accuracy): 0.793388429752
[[11  0  0]
 [56 11  0]
 [ 3 40  0]]
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  

Train on 109 samples, validate on 12 samples
Epoch 1/1000
109/109 [==============================] - 2s 19ms/step - loss: 1.6855 - acc: 0.4312 - val_loss: 1.1012 - val_acc: 0.4167
Epoch 2/1000
109/109 [==============================] - 0s 2ms/step - loss: 1.0350 - acc: 0.5229 - val_loss: 1.0420 - val_acc: 0.4167
Epoch 3/1000
109/109 [==============================] - 0s 2ms/step - loss: 1.1595 - acc: 0.2844 - val_loss: 0.9155 - val_acc: 0.8333
Epoch 4/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.9823 - acc: 0.6147 - val_loss: 0.7628 - val_acc: 0.4167
Epoch 5/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.9934 - acc: 0.5505 - val_loss: 0.7857 - val_acc: 0.5833
Epoch 6/1000
109/109 [==============================] - 0s 1ms/step - loss: 1.2801 - acc: 0.4404 - val_loss: 0.8124 - val_acc: 0.5833
Epoch 7/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.8317 - acc: 0.6972 - val_loss: 1.0514 - val_acc: 0.4167
Epoch 8/1000
109

109/109 [==============================] - 0s 1ms/step - loss: 0.5874 - acc: 0.8257 - val_loss: 0.1719 - val_acc: 1.0000
Epoch 62/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.2545 - acc: 0.8716 - val_loss: 0.0750 - val_acc: 1.0000
Epoch 63/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.4132 - acc: 0.8165 - val_loss: 0.1719 - val_acc: 1.0000
Epoch 64/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.3312 - acc: 0.8807 - val_loss: 0.1471 - val_acc: 1.0000
Epoch 65/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.2615 - acc: 0.8991 - val_loss: 1.0510 - val_acc: 0.6667
Epoch 66/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.4038 - acc: 0.8532 - val_loss: 0.0956 - val_acc: 1.0000
Epoch 67/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.3192 - acc: 0.8807 - val_loss: 0.1900 - val_acc: 1.0000
Epoch 68/1000
109/109 [==============================] - 0s 1ms/step 

109/109 [==============================] - 0s 2ms/step - loss: 0.1031 - acc: 0.9450 - val_loss: 0.0359 - val_acc: 1.0000
Epoch 122/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.2691 - acc: 0.8807 - val_loss: 0.0808 - val_acc: 1.0000
Epoch 123/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.1071 - acc: 0.9633 - val_loss: 2.4382 - val_acc: 0.6667
Epoch 124/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.1395 - acc: 0.9174 - val_loss: 0.0258 - val_acc: 1.0000
Epoch 125/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.1043 - acc: 0.9633 - val_loss: 0.6074 - val_acc: 0.8333
Epoch 00125: early stopping
Fold Score (accuracy): 0.816666666667
[[10  0  0]
 [69  1  0]
 [11 29  0]]
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  

In [160]:
#augX_train.shape
#augX_train[:,9].shape
#plt.imshow(augX_train[:,9])
#datagen.flow(augX_train[:0], batch_size=1)
datagen.flow(augX_train[:0], batch_size=10)

In [ ]:
plt.imshow(augimX_train)

In [ ]:
print(len(y_train))
print(y_train)
by_train = y_train[:,0] == 1
by_val = y_val[:,0] == 1
by_test = y_test[:,0] == 1
print(len(by_train))
print(Counter(by_train).values())
print(by_train)
augX_train, augX_val, augX_test = X_train[by_train], X_val[by_val], X_test[by_test]
print(len(augX_train))
print(augX_train)


In [ ]:
X.shape

In [ ]:
f = cv2.imread('/home/diam/Desktop/HER2_data_aug_0/FDA_0_aug/aug_8_7339873.tif')



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(f)

In [ ]:
fhsv = cv2.cvtColor(f,cv2.COLOR_BGR2HSV)

In [ ]:
fhsv > im.tif

In [ ]:
cv2.imwrite('imtest.tif', fhsv)

In [ ]:
from libtiff import TIFF